In [4]:
import pandas as pd
# Define the file path to the CSV file that needs to be read
file_path = 'UoB_Set01_2025-01-02tapes.csv'

# Set header=None and manually specify the column names as 'Time', 'Price', 'Size'
df = pd.read_csv(file_path, header=None, names=['Time', 'Price', 'Size'])

# Using the apply method to calculate the weighted average price and total size
weighted_df = df.groupby('Time').apply(lambda x: pd.Series({
    'Weighted_Price': (x['Price'] * x['Size']).sum() / x['Size'].sum(),
    'Total_Size': x['Size'].sum(),
    'Order_Number': x.shape[0]
})).reset_index()

weighted_df[:20]


C:\Users\lenovo\AppData\Local\Temp\ipykernel_14404\1440805371.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_df = df.groupby('Time').apply(lambda x: pd.Series({


,Time,Weighted_Price,Total_Size,Order_Number
0,10.881,267.000000,1.0,1.0
1,11.067,269.000000,1.0,1.0
2,11.222,267.000000,2.0,1.0
3,12.338,270.000000,2.0,1.0
4,13.733,267.000000,3.0,1.0
5,18.321,265.000000,2.0,1.0
6,19.654,263.666667,3.0,3.0
7,21.886,264.000000,1.0,1.0
8,22.258,264.000000,4.0,1.0
9,22.382,263.000000,1.0,1.0


In [5]:
# Save the processed data to a new CSV file
output_file_path = 'Clean_UoB_Set01_2025-01-02tapes.csv'
weighted_df.to_csv(output_file_path, index=False)
